# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
import os
from urllib.request import urlretrieve
import zipfile
import numpy as np
import cv2
import tensorflow as tf
import pickle
from tqdm import tqdm
from batch_iter import batch_iterator
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

In [ ]:
#Download and unzip the data set
def download(url, file):
    """
    Download file from <url>
    :param url: URL to file
    :param file: Local file path
    """
    if not os.path.isfile(file):
        print('Downloading ' + file + '...')
        urlretrieve(url, file)
        print('Download Finished')

download('https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip', 'traffic-signs-data.zip')
f = zipfile.ZipFile('traffic-signs-data.zip')
f.extractall('traffic-signs-data')  
f.close()

In [ ]:
# TODO: Fill this in based on where you saved the training and testing data
training_file = 'traffic-signs-data/train.p'
validation_file='traffic-signs-data/valid.p'
testing_file = 'traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

assert(X_train.shape[0]==y_train.shape[0])
assert(X_valid.shape[0]==y_valid.shape[0])
assert(X_test.shape[0]==y_test.shape[0])

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = y_train.shape[0]

# TODO: Number of validation examples
n_validation = y_valid.shape[0]

# TODO: Number of testing examples.
n_test = y_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1]

# TODO: How many unique classes/labels there are in the dataset.
sign_classes, class_index, class_counts = np.unique(y_train, return_index=True, return_counts=True)
n_classes = sign_classes.shape[0]

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Show one example of each class
sign = 0
while sign <= n_classes:
    fig = plt.figure(figsize=(10,1))
    fig.subplots_adjust(left = 0, right = 1, bottom = 0, top = 1, hspace = 0.05, wspace = 0.05)
    show_indices = class_index[sign: sign+10]
    for i in range(len(show_indices)):
        axis = fig.add_subplot(1, 10, i+1, xticks=[], yticks=[])
        axis.imshow(X_train[show_indices[i]])
    sign += 10
plt.show    

In [ ]:
# histogram of label frequency
plt.bar(np.arange(n_classes), class_counts, align='center')
plt.show()
print('The category of %s contains the most examples of %s' % (np.argmax(class_counts),np.max(class_counts)))
print('The category of %s contains the least examples of %s' % (np.argmin(class_counts),np.min(class_counts)))

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
#Transform the images by translating, scaling, rotating and changing brightness for the next augmentation        
def transform(Xa, p = 0.6, intensity = 0.5):
    """
    Applies a pipeline of randomised transformations for data augmentation.
    p         : Probability of augmenting a single example, should be in a range of [0, 1] .
                Defines data augmentation coverage.
    intensity : Augmentation intensity, should be in a [0, 1] range.        
    """           
    batch_size = Xa.shape[0]
    cols = rows = Xa.shape[1]
    
    def random_translate(Xa):
        """
        Applies random translation to a random subset of images
        """
        # allow translation up to px pixels in x and y directions
        px = 5        
        for i in np.random.choice(batch_size, int(batch_size * p), replace=False):            
            dx,dy = np.random.randint(-px * intensity, px * intensity, 2)
            M = np.float32([[1,0,dx],[0,1,dy]])
            Xa[i] = cv2.warpAffine(Xa[i],M,(cols,rows))    
        return Xa

    def random_scaling(Xa):
        """
        Applies random scaling to a random subset of images
        """
        # transform limits
        px = 5        
        for i in np.random.choice(batch_size, int(batch_size * p), replace=False):
            ptx = np.random.randint(-px * intensity, px * intensity)
            # ending locations
            pts1 = np.float32([[ptx,ptx],[rows-ptx,ptx],[ptx,cols-ptx],[rows-ptx,cols-ptx]])
            # starting locations (4 corners)
            pts2 = np.float32([[0,0],[rows,0],[0,cols],[rows,cols]])
            M = cv2.getPerspectiveTransform(pts1,pts2)
            Xa[i] = cv2.warpPerspective(Xa[i],M,(rows,cols))       
        return Xa

    def random_rotate(Xa):    
        # random rotation coefficients
        rx = 5
        for i in np.random.choice(batch_size, int(batch_size * p), replace=False):
            rtx = np.random.randint(-rx * intensity, rx * intensity) 
            M = cv2.getRotationMatrix2D((cols/2,rows/2),rtx,1)
            Xa[i] = cv2.warpAffine(Xa[i],M,(cols,rows))
        return Xa

    def random_brightness(Xa):
        for i in np.random.choice(batch_size, int(batch_size * p), replace=False):
            btx = np.absolute(np.random.normal(1,0.3*intensity))
            for row in range(rows):
                for col in range(cols):
                    Xa[i, row, col, 0] = np.minimum(Xa[i, row, col, 0] * btx, 255)
                    Xa[i, row, col, 1] = np.minimum(Xa[i, row, col, 1] * btx, 255)
                    Xa[i, row, col, 2] = np.minimum(Xa[i, row, col, 2] * btx, 255)                    
        return Xa    
    
    Xa = random_brightness(random_rotate(random_scaling(random_translate(Xa))))
        
    return Xa


In [ ]:
#Create balanced training set by augmenting classes with small amount of examples
def data_balancing(X, y, n_target = None):
    """
    Augment the dataset by transforming the existing images. Make every class contains the same number of examples
    
    Parameters
    ----------
    X             : Image dataset
    y             : label dataset
    p             : Probability of augmenting a single example
    aug_intensity : Intensity of augmentation, must be in [0, 1] range.
    n_target      : Indicate target number of each class after augmentation. If is none, the number of each class will be augmented to the 
                    maximum amoung all classes
                    
    Returns
    -------
    A tuple of X and y.    
    """
    sign_classes, class_counts = np.unique(y, return_counts = True)
    n_classes = sign_classes.shape[0]
    Max_c = max(class_counts)
    Target = Max_c if n_target is None else n_target
    
    X_aug = np.empty([0, X.shape[1], X.shape[2], X.shape[3]], dtype = np.float32)
    y_aug = np.empty([0], dtype = y.dtype)
    
    for i in tqdm(range(n_classes)):
        #First copy the existing data for certain class i
        X_ori = X[y==i].astype(np.float32)
        y_ori = y[y==i]
        X_aug = np.append(X_aug, X_ori, axis = 0)
        
        for j in range((Target // class_counts[i])-1):
            x_batch = transform(X_ori)
            X_aug = np.append(X_aug, x_batch, axis = 0)
            
        batchsize = Target % class_counts[i]
        if batchsize:        
            batch_aug = batch_iterator(batchsize)
            for x_batch, _ in batch_aug(X_ori, y_ori):
                x_batch = transform(x_batch)
                X_aug = np.append(X_aug, x_batch, axis = 0)
                break
        
        added = X_aug.shape[0] - y_aug.shape[0]
        y_aug = np.append(y_aug, np.full((added), i, dtype = int))
        
    return (X_aug.astype(np.uint8), y_aug)

In [ ]:
## Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle

def preprocess_dataset(X, y = None, shuffl = True):
    """
    Performs feature scaling, normalization, enhance the contrast and shuffling.
    
    Parameters
    ----------
    X: array of images, shape=[None, width, heights, RGB]
    y: array of labels
    Returns
    -------
    A tuple of X, y
    """
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(6,6))
    X_out = np.empty([0, X.shape[1], X.shape[2]], dtype=np.float32)
    for i in tqdm(range(X.shape[0])):
        #Greyscale
        img_t = cv2.cvtColor(X[i], cv2.COLOR_BGR2GRAY) 
        #Enhance the contrast of images
        img_t = clahe.apply(img_t)
        #Normalization 
        img_t = normalize(img_t)
        img_t = img_t.reshape(1,32,32)
        X_out = np.append(X_out,img_t, axis=0)
    X = X_out
    #One-hot encoding
    if y is not None:
        y = np.eye(n_classes)[y]
        #shuffle
        if shuffl:
            X, y = shuffle(X, y)
    X = X.reshape(X.shape + (1,))
   
    return X, y


In [ ]:
#Preprocess the training, validation and test set and store them in respective files
import pickle

trainset_aug_file = "traffic-signs-data/train_aug.p"
train_processed_file = "traffic-signs-data/train_processed.p"
validset = "traffic-signs-data/valid_processed.p"
testset = "traffic-signs-data/test_processed.p"

X_train, y_train = data_balancing(X_train, y_train)
sign_classes, class_counts = np.unique(y_train, return_counts=True)
plt.bar(np.arange(n_classes), class_counts, align='center')
plt.show()
print("Number of training examples after augmenting =", X_train.shape[0])
pickle.dump({
    "features" : X_train,
    "labels" : y_train
},open(trainset_aug_file, "wb"))
print("Augmented data saved in", trainset_aug_file)

X_train, y_train = preprocess_dataset(X_train,y_train)
pickle.dump({
    "features" : X_train,
    "labels" : y_train
},open(train_processed_file, "wb"))
print("Processed data saved in", train_processed_file)

X_valid, y_valid = preprocess_dataset(X_valid, y_valid)
pickle.dump({
    "features" : X_valid,
    "labels" : y_valid
},open(validset, "wb"))
print("Augmented data saved in", validset)

X_test, y_test = preprocess_dataset(X_test,y_test)
pickle.dump({
    "features" : X_test,
    "labels" : y_test
},open(testset, "wb"))
print("Augmented data saved in", testset)

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
from tensorflow.contrib.layers import flatten
def fully_connected(input, size):
    """
    Performs a single fully connected layer pass, e.g. returns `input * weights + bias`.
    """
    weights = tf.get_variable( 'weights', 
        shape = [input.get_shape()[1], size],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [size],
        initializer = tf.constant_initializer(0.0)
      )
    return tf.matmul(input, weights) + biases

def fully_connected_relu(input, size):
    return tf.nn.relu(fully_connected(input, size))

def conv_relu(input, kernel_size, depth):
    """
    Performs a single convolution layer pass.
    """
    weights = tf.get_variable( 'weights', 
        shape = [kernel_size, kernel_size, input.get_shape()[3], depth],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [depth],
        initializer = tf.constant_initializer(0.0)
      )
    conv = tf.nn.conv2d(input, weights,
        strides = [1, 1, 1, 1], padding = 'SAME')
    return tf.nn.relu(conv + biases)

def pool(input, size):
    """
    Performs a max pooling layer pass.
    """
    return tf.nn.max_pool(
        input, 
        ksize = [1, size, size, 1], 
        strides = [1, size, size, 1], 
        padding = 'SAME'
    )

def Lenet(input, is_training):
    """
    Performs a full model pass.
    
    Parameters
    ----------
    input         : Tensor
                    NumPy array containing a batch of examples.
    is_training   : Tensor of type tf.bool
                    Flag indicating if we are training or not (e.g. whether to use dropout).
                    
    Returns
    -------
    Tensor with predicted logits.
    """
    # Convolutions

    with tf.variable_scope('conv1'):
        conv1 = conv_relu(input, kernel_size = 5, depth = 32) 
    with tf.variable_scope('pool1'): 
        pool1 = pool(conv1, size = 2)
        pool1 = tf.cond(is_training, lambda: tf.nn.dropout(pool1, keep_prob = 0.8), lambda: pool1)
    with tf.variable_scope('conv2'):
        conv2 = conv_relu(pool1, kernel_size = 5, depth = 64)
    with tf.variable_scope('pool2'):
        pool2 = pool(conv2, size = 2)
        pool2 = tf.cond(is_training, lambda: tf.nn.dropout(pool2, keep_prob = 0.7), lambda: pool2)
    with tf.variable_scope('conv3'):
        conv3 = conv_relu(pool2, kernel_size = 5, depth = 128)
    with tf.variable_scope('pool3'):
        pool3 = pool(conv3, size = 2)
        pool3 = tf.cond(is_training, lambda: tf.nn.dropout(pool3, keep_prob = 0.6), lambda: pool3)
    
    # Fully connected
    
    # 1st stage output
    pool1 = pool(pool1, size = 4)
    pool1 = flatten(pool1)
    
    # 2nd stage output
    pool2 = pool(pool2, size = 2)
    pool2 = flatten(pool2)    
    
    # 3rd stage output
    pool3 = flatten(pool3)
    
    flattened = tf.concat(1, [pool1, pool2, pool3])
    
    with tf.variable_scope('fc4'):
        fc4 = fully_connected_relu(flattened, size = 1024)
        fc4 = tf.cond(is_training, lambda: tf.nn.dropout(fc4, keep_prob = 0.5), lambda: fc4)
    with tf.variable_scope('out'):
        logits = fully_connected(fc4, size = 43)
    return logits
    

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
#Load training, validation and test data from files

train_processed_file = "traffic-signs-data/train_processed.p"
with open(train_processed_file, mode='rb') as f:
    train = pickle.load(f)    
X_train, y_train = train['features'], train['labels']

valid_processed_file = "traffic-signs-data/valid_processed.p"
with open(valid_processed_file, mode='rb') as f:
    valid = pickle.load(f)    
X_valid, y_valid = valid['features'], valid['labels']

test_processed_file = "traffic-signs-data/test_processed.p"
with open(test_processed_file, mode='rb') as f:
    test = pickle.load(f)    
X_test, y_test = test['features'], test['labels']


In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
EPOCHS = 100
BATCHSIZE = 128
learn_rate = 0.001
l2_reg = 0.001
save_path = './lenet'

graph = tf.Graph()
with graph.as_default():
    tf_x_batch = tf.placeholder(tf.float32, shape = (None, X_train.shape[1], X_train.shape[2], 1))
    tf_y_batch = tf.placeholder(tf.int32, shape = (None))
    is_training = tf.placeholder(tf.bool)
    
    logits = Lenet(tf_x_batch, is_training)
    if l2_reg:
        with tf.variable_scope('fc4', reuse = True):
            l2_loss = tf.nn.l2_loss(tf.get_variable('weights'))
    else:
        l2_loss = 0
        
    predictions = tf.nn.softmax(logits)
    softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_y_batch)
    loss = tf.reduce_mean(softmax_cross_entropy) + l2_loss
    
    optimizer = tf.train.AdamOptimizer(learning_rate = learn_rate).minimize(loss)
    
    def evaluate(X, y, sess):
        if X.shape[3] != 1:
            try:
                X = X = 0.299 * X[:, :, :, 0] + 0.587 * X[:, :, :, 1] + 0.114 * X[:, :, :, 2]
                X = X.reshape(X.shape + (1,))
            except ValueError:
                print('The color channel of images is %d' % (X.shape[3]))
                
        acc = np.empty([0, n_classes])
        los = np.empty([0])
        batch = batch_iterator(batchsize = BATCHSIZE)
        for x_batch, y_batch in batch(X, y):
            [acc_b, los_b] = sess.run([predictions, softmax_cross_entropy],feed_dict={
                tf_x_batch: x_batch,
                tf_y_batch: y_batch,
                is_training: False
            })
            acc = np.append(acc, acc_b, axis = 0)
            los = np.append(los, los_b, axis = 0)
        predict = np.argmax(acc,1)
        accuracy = 100.0 * np.sum(np.argmax(acc,1)==np.argmax(y,1))/acc.shape[0]
        loss = np.mean(los)
        return (accuracy, loss, predict)    
    
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    batch = batch_iterator(batchsize = BATCHSIZE)
    for i in range(EPOCHS):
        for x_batch, y_batch in batch(X_train, y_train):
             sess.run([optimizer], feed_dict = {
                tf_x_batch: x_batch,
                tf_y_batch: y_batch,
                is_training: True
            })
        
        train_accuracy, train_loss, _ = evaluate(X_train, y_train, sess)
        validation_accuracy,_ ,_= evaluate(X_valid, y_valid, sess)
        print("EPOCH {} ...".format(i+1))
        print("Train Accuracy = {:.3f}".format(train_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print("Loss = {:.3f}".format(train_loss))
        print()
        
    test_accuracy = evaluate(X_test, y_test, sess)
    print("Test Accuracy = {:.3f}".format(test_accuracy))
    saver.save(sess, save_path)
    print('Model saved')

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
X_new = np.empty([0, 32, 32, 3])
y_new = [ ]
test_images = os.listdir("test_examples/")
fig = plt.figure(figsize=(10,1))
fig.subplots_adjust(left = 0, right = 1, bottom = 0, top = 1, hspace = 0.05, wspace = 0.05)
for (image,i) in zip(test_images,range(5)):
    if 'jpg' in image:
        label = int(image[0:image.index('.')])
        y_new.append(label)
        #read image
        image = cv2.imread('test_examples/' + image)
        axis = fig.add_subplot(1, 10, 2*i+1, xticks=[], yticks=[])
        #show original image
        axis.imshow(image)
        #resize image to (32,32)
        image = cv2.resize(image,(32,32),interpolation=cv2.INTER_CUBIC)
        #store the image in new_img
        X_new= np.append(X_new, image.reshape((1,) + image.shape), axis = 0)
        #change to gray channel
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        axis = fig.add_subplot(1, 10, 2*i+2, xticks=[], yticks=[])
        axis.imshow(image, cmap="gray")
plt.show
y_new = np.array(y_new)
X_new, y_new = preprocess_dataset(X_new.astype(np.uint8), y_new, shuffl = False)

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.
from pandas.io.parsers import read_csv
signnames = read_csv("signnames.csv").values[:, 1]

with tf.Session(graph=graph) as sess:
    saver = tf.train.Saver()
    saver.restore(sess, save_path)
    
    acc, loss, predicts = evaluate(X_new, y_new, sess)
    accu, losses, _ = evaluate(X_train, y_train, sess)
    print(accu, losses)
    for label, predict,i in zip(np.argmax(y_new, 1), predicts, range(len(y_new))):
        print('Example %d is %s, and predicted as %s' % (i, signnames[label], signnames[predict]))

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.
print('The accuracy of 5 new images is',str(acc),'%')

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
with tf.Session(graph=graph) as sess:
    saver = tf.train.Saver()
    saver.restore(sess, save_path)
    
    [top_5_softmax, indices] = sess.run(tf.nn.top_k(predictions, 5), feed_dict = {
            tf_x_batch: X_new,
            is_training: False        
    })
for i in range(X_new.shape[0]):
    plt.figure(figsize = (6, 2))
    gs = gridspec.GridSpec(1, 3)
    
    plt.subplot(gs[0,0])
    plt.imshow(X_new[i].reshape(32,32), cmap='gray')
    plt.axis('off')
    
    plt.subplot(gs[0,1:])
    plt.barh(np.arange(5)+.5, top_5_softmax[i], align = 'center')
    plt.yticks(np.arange(5)+.5, signnames[indices[i]])
    plt.tick_params(axis='both', which='both', labelleft='off', labelright='on', labeltop='off', labelbottom='off')
    plt.savefig('top5 image%d' %(i))
    
    plt.show

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={tf_x_batch : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")